In [6]:
#Dependencies
import pandas as pd
import requests
import json
from config import api_key


In [7]:
states_abbrev_file_path = "Resources/states_abbrev.csv"

states_abbrev_df = pd.read_csv(states_abbrev_file_path)
states_abbrev_df

,State,Abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [13]:
# API Call for covid data 
covid_base_url = "https://api.covidtracking.com/v1/states/daily.json"
response = requests.get(covid_base_url).json()

#print(json.dumps(response, indent=4))

# pull the data from a source df to a target df by creating a new df

covid_df = pd.DataFrame(response, columns=['date', 'state', 'positive', 'death', 'hospitalized', 'totalTestResults'])
covid_df.fillna(0)
covid_df.head()

,date,state,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [9]:
for x, y in covid_df.iterrows():
    org_date = str(y.get(key='date'))
   
    year_s = org_date[0:4]
    month_s = org_date[4:6]
    day_s = org_date[6:8]
    formatted_date = f"{year_s}-{month_s}-{day_s}"
    
        
covid_df['Formatted Date'] = formatted_date


covid_df.head()

        

,date,state,positive,death,hospitalized,totalTestResults,Formatted Date
0,20201021,AK,12644.0,68.0,NaN,547140.0,2020-01-22
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0,2020-01-22
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0,2020-01-22
3,20201021,AS,0.0,0.0,NaN,1616.0,2020-01-22
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0,2020-01-22


In [11]:
# API Call for flights data 

url = f"http://api.aviationstack.com/v1/flights?access_key={api_key}&"
query_date = ['2020-10-01','2020-10-02','2020-10-03' ]
query_iata = ['JFK', 'SAN']
value_list = []

for q_date in query_date:
    for q_iata in query_iata:
        flights_base_url = f"{url}flight_date={q_date}&arr_iata={q_iata}&flight_status=landed&limit=1"
        flights_response = requests.get(flights_base_url).json()
        print(flights_response)
        landed_flights = flights_response['pagination']['total']
        state = flights_response['data'][0]['arrival']['timezone']
        state = state.split('/')[1]
        state = state.replace('_', ' ')
       
        value_list.append({'Flight Date': q_date, 'State': state, 'IATA':q_iata, 'Total Landed Flights': landed_flights})
        
flights_df = pd.DataFrame(value_list)   
flights_df

{'error': {'code': 'function_access_restricted', 'message': 'Your current subscription plan does not support this API function.'}}
{'error': {'code': 'function_access_restricted', 'message': 'Your current subscription plan does not support this API function.'}}
{'error': {'code': 'function_access_restricted', 'message': 'Your current subscription plan does not support this API function.'}}
{'error': {'code': 'function_access_restricted', 'message': 'Your current subscription plan does not support this API function.'}}
{'error': {'code': 'function_access_restricted', 'message': 'Your current subscription plan does not support this API function.'}}
{'error': {'code': 'function_access_restricted', 'message': 'Your current subscription plan does not support this API function.'}}


In [ ]:
#df1.merge(df2,on='name').merge(df3,on='name')
merged_df = states_abbrev_df.merge(flights_df, on='State')
merged_df

In [14]:
#merge covid data and flights data
merge_date_df = merged_df.merge(covid_df, on = 'Date')

#in the dataframe i should have ['Date', 'State', 'Infection Rate', 'Flights landed'] merging by dates

merge_date_df["Month"] = merge_date_df['Date'].dt.month
total_infection = merge_data_df.groupby(["Month"]).sum()["Infection Rate"]
landing_flights = merge_data_df.groupby(["Month"]).sum()["Flights landed"]
final_df = pd.Dataframe({"Total Infection": total_infection, "Landed Flights": landing_flights})
final_df
final_df.fillna(0)

NameError: name 'merged_df' is not defined